<a href="https://colab.research.google.com/github/leandro-jm/azure-openai-read-pdf/blob/main/azure_openai_read_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install azure-search-documents==11.4.0b6
!pip install azure-identity
!pip install langchain==0.0.248
!pip install openai==0.27.8
!pip install python-dotenv==1.0.0
!pip install uvicorn==0.23.2
!pip install fastapi==0.100.1
!pip install tiktoken
!pip install pypdf

# New Section

In [ ]:
import os
import openai
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.agents import initialize_agent, Tool, AgentType


load_dotenv(verbose=True)

openai.api_type = "azure"
openai.api_version = "2023-03-15-preview"
openai.api_key = "07e7409b65c84df68f1aabd777ed172a"
openai.api_base = "https://openai-xpe-001.openai.azure.com/"
model: str = "text-embedding-ada-002"

In [ ]:
openai_api_key = "07e7409b65c84df68f1aabd777ed172a"
openai_env= "https://openai-xpe-001.openai.azure.com/"
embeddings = OpenAIEmbeddings(deployment='text-embedding-ada-002', openai_api_key=openai_api_key, model="text-embedding-ada-002")

embeddings: OpenAIEmbeddings = OpenAIEmbeddings(openai_api_key=openai_api_key, deployment=model, model=model, chunk_size=1, openai_api_base="https://openai-xpe-001.openai.azure.com/", openai_api_type="azure" )
index_name: str = "xpe-vector-demo"
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint="https://cog-xpe-openai.search.windows.net",
    azure_search_key="BVIMUpgqTQoNplDPHhYbPzYCm51YUUUUblbb63X8uFAzSeC1LfNF",
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

vector_store = AzureSearch(
    azure_search_endpoint="https://cog-xpe-openai.search.windows.net",
    azure_search_key="BVIMUpgqTQoNplDPHhYbPzYCm51YUUUUblbb63X8uFAzSeC1LfNF",
    index_name="xpe-vector-demo",
    embedding_function=embeddings.embed_query,
)

In [ ]:
loader = PyPDFDirectoryLoader("pdfs")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

vector_store.add_documents(documents=docs);

In [ ]:
llm = ChatOpenAI(engine='gpt-4', verbose=False,openai_api_key=openai_api_key)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
    return_source_documents=True
)

                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [ ]:
query = "Explique para uma criança de 10 anos o que o gerente de TI Faz e o que é a empresa em questão ?"
result = qa({"query": query})
result["result"]

'O Gerente de TI, ou Tecnologia da Informação, é como um super-herói dos computadores e da tecnologia em uma empresa. Ele garante que todos os computadores, programas e sistemas da empresa estejam funcionando bem e sem problemas. Se algo der errado, como um computador que não liga ou um programa que não funciona, é o trabalho do Gerente de TI resolver. Ele também ajuda a criar novas maneiras de usar a tecnologia para que a empresa possa fazer seu trabalho melhor e mais rápido.\n\nA empresa em questão, Contoso Electronics, é como uma grande loja de brinquedos, mas para equipamentos eletrônicos. Eles criam e vendem todos os tipos de produtos eletrônicos, como computadores, telefones, TVs e muito mais. Eles também pensam em novos produtos e maneiras de melhorar os produtos que já possuem. Assim como em uma loja de brinquedos, há muitas pessoas diferentes trabalhando em coisas diferentes para garantir que tudo funcione bem.'

In [ ]:
def reverse_string(string):
    return string[::-1]

print(reverse_string("abcd123"))

321dcba
